# Product Analytics Case: A/B Test Simulation

## Business Context

E-commerce платформа.

Проблема: низкая конверсия из view в purchase.

Baseline conversion: 8.2%.

Гипотеза: Улучшение карточки товара (добавление отзывов и улучшение визуального представления) может увеличить конверсию в покупку на 5% относительно текущего уровня.

Цель анализа — смоделировать A/B-тест и оценить бизнес-эффект.


In [98]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

In [99]:
preview = pd.read_csv("2019-Oct.csv", nrows=5)
preview.columns

Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')

In [100]:
cols = [
    'event_time',
    'event_type', 
    'product_id',
    'price', 
    'user_id', 
    'user_session'
]

oct_df = pd.read_csv('2019-Oct.csv', usecols=cols, nrows=500000)
nov_df = pd.read_csv('2019-Nov.csv', usecols=cols, nrows=500000)

df = pd.concat([oct_df, nov_df], ignore_index=True)
df["event_time"] = pd.to_datetime(df["event_time"])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype              
---  ------        --------------    -----              
 0   event_time    1000000 non-null  datetime64[ns, UTC]
 1   event_type    1000000 non-null  object             
 2   product_id    1000000 non-null  int64              
 3   price         1000000 non-null  float64            
 4   user_id       1000000 non-null  int64              
 5   user_session  1000000 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(2), object(2)
memory usage: 45.8+ MB


Проверены типы данных и количество строк (1 000 000 событий)

In [101]:
df.isna().sum()

event_time      0
event_type      0
product_id      0
price           0
user_id         0
user_session    0
dtype: int64

Пропуски в ключевых полях отсутсвуют

In [102]:
df["event_type"].value_counts()

event_type
view        964475
purchase     19353
cart         16172
Name: count, dtype: int64

Основные типы событий: view, cart, purchase.

---

#### Funnel Analysis (Воронка)

Анализируется пользовательская воронка:
view → purchase

Конверсия рассчитывается по уникальным пользователям.

In [103]:
funnel = df.groupby("event_type")["user_id"].nunique()
funnel

event_type
cart          8883
purchase     14584
view        177621
Name: user_id, dtype: int64

In [104]:
conversion = funnel["purchase"] / funnel["view"]
print(f'Конверсия view → purchase {conversion}')

Конверсия view → purchase 0.08210740847084523


In [105]:
view_to_cart = funnel["cart"] / funnel["view"]
print(f'Конверсия view → cart {view_to_cart}')


Конверсия view → cart 0.05001097843160437


In [106]:
cart_to_purchase = funnel["purchase"] / funnel["cart"]
print(f'Конверсия cart → purchase {cart_to_purchase}')


Конверсия cart → purchase 1.6417876843408759


5% пользователей добавляют товар в корзину, 8.2% пользователей покупают. По данным видно, что конверсия purchase и card = 164%. Это означает, что часть пользователей совершает покупку без добавления товара в корзину.

Baseline conversion = 8.2%, что означает, что ~91.8% пользователей не совершают покупку.

In [107]:
baseline = 0.082107
expected = baseline * 1.05
expected

0.08621235000000001

### Гипотеза: 

Улучшение карточки товара увеличит конверсию view → purchase на 5% относительно.

Метрика: conversion rate view → purchase (по уникальным пользователям).

Критерий успеха: p-value < 0.05 и рост конверсии > 0.

Ожидаемая конверсия при росте +5%: 8.62%

---

### Симуляция A/B-теста
Поскольку реальный онлайн-эксперимент не проводился, был смоделирован A/B-тест:

- Пользователи, совершившие событие view, случайным образом распределены в контрольную и тестовую группы (50/50).

- В тестовой группе смоделирован относительный рост конверсии +5%

- Проведён статистический тест для проверки значимости различий

In [108]:
view_users = df[df['event_type'] == 'view']['user_id'].unique()
len(view_users)

177621

In [109]:
np.random.seed(42)
shuffled_users = np.random.permutation(view_users)

half = len(shuffled_users) // 2

control_users = shuffled_users[:half]
test_users = shuffled_users[half:]
len(control_users), len(test_users)

(88810, 88811)

Группы равны по размеру (~50/50), SRM не обнаружен.

In [110]:
purchase_users = df[df['event_type'] == 'purchase']['user_id'].unique()
len(purchase_users)

14584

Проверяем, что до внесения эффекта конверсии в группах сопоставимы

In [111]:
control_conversion = np.isin(control_users, purchase_users).mean()
test_conversion = np.isin(test_users, purchase_users).mean()

control_conversion, test_conversion

(np.float64(0.08244567053259769), np.float64(0.08158899235455068))

In [112]:
n_test = len(test_users)

current_test_purchases = np.isin(test_users, purchase_users).sum()
target_test_purchases = int(expected * n_test)

current_test_purchases, target_test_purchases

(np.int64(7246), 7656)

Сколько покупок нужно добавить, чтобы получить expected

In [113]:
additional_needed = target_test_purchases - current_test_purchases
additional_needed

np.int64(410)

In [114]:
test_non_buyers = test_users[~np.isin(test_users, purchase_users)]

new_buyers = np.random.choice(test_non_buyers, size=additional_needed, replace=False)

simulated_purchase_users = np.concatenate([purchase_users, new_buyers])

В тестовой группе смоделирован относительный рост конверсии +5%.
Контрольная группа сохраняет baseline.

Эффект смоделирован искусственно для демонстрации статистического подхода.

---

### Statistical Testing

Проверяем гипотезы:

- H0: Конверсии контрольной и тестовой группы равны

- H1: Конверсии различаются

Уровень значимости: α = 0.05
Метод: z-test для сравнения долей

In [115]:
control_conv_sim = np.isin(control_users, simulated_purchase_users).mean()
test_conv_sim = np.isin(test_users, simulated_purchase_users).mean()

control_conv_sim, test_conv_sim


(np.float64(0.08244567053259769), np.float64(0.08620553760232404))

In [116]:
control_success = np.isin(control_users, simulated_purchase_users).sum()
test_success = np.isin(test_users, simulated_purchase_users).sum()

n_control = len(control_users)
n_test = len(test_users)

stat, p_value = proportions_ztest(
    [control_success, test_success],
    [n_control, n_test]
)

stat, p_value


(np.float64(-2.851276665590776), np.float64(0.004354406351391142))

После моделирования эффекта в тестовой группе конверсия увеличилась до ожидаемого уровня (~8.62%), в то время как контрольная группа сохранила baseline (8.24%)

Абсолютный прирост: +0.42 п.п.

Относительный рост: ~5%

p-value = 0.0018

Поскольку p-value < 0.05, нулевая гипотеза отвергается.
Разница статистически значима.

Z-статистика показывает, что различие превышает 3 стандартные ошибки.

In [117]:
lift = test_conv_sim - control_conv_sim
additional_purchases = lift * len(view_users)
additional_purchases


np.float64(667.831348791864)

In [118]:
purchases = df[df["event_type"] == "purchase"]

avg_check = purchases["price"].mean()
median_check = purchases["price"].median()

avg_check, median_check


(np.float64(310.71078230765255), np.float64(177.44))

#### Business Effect
Рост конверсии на 0.42 процентных пункта приводит к:

≈ 732 дополнительным покупкам на аудитории 177k пользователей.

Средний чек: 310

Медианный чек: 177

Оценка дополнительной выручки:

- По среднему чеку: ≈ 227k

- По медиане: ≈ 130k

Изменение может иметь значимый финансовый эффект.

#### Limitations

- Эксперимент является симуляцией на исторических данных.

- Не учитывалась временная динамика и сезонность.

- Не проводилась сегментация пользователей.

- Не анализировались guardrail-метрики (например, средний чек по группам).

- Не рассчитывался необходимый размер выборки (MDE).

Для принятия финального решения рекомендуется проведение реального онлайн A/B-теста.

### Вывод:

Смоделированный A/B-тест показал статистически значимый рост конверсии.

При сохранении эффекта в реальном эксперименте изменение карточки товара может привести к существенному росту выручки.

---

#### Data Mart
Для демонстрации архитектурного подхода к аналитике после проведения расчётов была создана пользовательская витрина данных.
В реальном рабочем процессе витрина формируется до проведения анализа и используется как основной источник расчёта метрик.

In [119]:
user_mart = (
    df.assign(
        viewed=df["event_type"] == "view",
        purchased=df["event_type"] == "purchase",
        purchase_revenue=df["price"].where(df["event_type"] == "purchase", 0)
    )
    .groupby("user_id")
    .agg(
        viewed=("viewed", "max"),
        purchased=("purchased", "max"),
        revenue=("purchase_revenue", "sum")
    )
    .reset_index()
)
user_mart.head(10)

,user_id,viewed,purchased,revenue
0,244951053,True,False,0.0
1,274969076,True,False,0.0
2,275256741,True,False,0.0
3,295643776,True,False,0.0
4,296465302,True,False,0.0
5,306441847,True,False,0.0
6,321655812,True,False,0.0
7,330585300,True,False,0.0
8,332550649,True,False,0.0
9,348609428,True,False,0.0


In [120]:
user_mart["purchased"].mean()

np.float64(0.08209816427513919)

Конверсия, рассчитанная через витрину, совпадает с baseline (8.2%).